In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
    .builder\
    .appName("twitter_silver")\
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/05/30 15:43:25 WARN Utils: Your hostname, BIRIBA resolves to a loopback address: 127.0.1.1; using 172.19.51.55 instead (on interface eth0)
23/05/30 15:43:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/30 15:43:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_tweet = spark.read.json("/home/raf/pipeline-transform/datalake/silver/twitter_datascience/tweet")

In [4]:
df_tweet.printSchema()

root
 |-- author_id: string (nullable = true)
 |-- conversation_id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- id: string (nullable = true)
 |-- like_count: long (nullable = true)
 |-- quote_count: long (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- text: string (nullable = true)
 |-- process_date: date (nullable = true)



In [5]:
from pyspark.sql import functions as f

In [6]:
tweet_conversas = df_tweet.alias("tweet")\
        .groupBy(f.to_date("created_at").alias("created_date"))\
        .agg(
            f.countDistinct("author_id").alias("n_tweets"),
            f.sum("like_count").alias("n_like"),
            f.sum("quote_count").alias("n_quote"),
            f.sum("reply_count").alias("n_reply"),
            f.sum("retweet_count").alias("n_retweet")
        ).withColumn("weekday", f.date_format("created_date", "E"))


In [7]:
tweet_conversas.orderBy("created_date").show()

+------------+--------+------+-------+-------+---------+-------+
|created_date|n_tweets|n_like|n_quote|n_reply|n_retweet|weekday|
+------------+--------+------+-------+-------+---------+-------+
|  2023-05-24|      19|  4614|   6876|   6186|     5574|    Wed|
|  2023-05-25|      18|  5652|   5940|   4932|     6150|    Thu|
|  2023-05-26|      40| 14425|  14055|  13345|    12315|    Fri|
|  2023-05-27|      33|  5433|   5898|   6147|     5742|    Sat|
|  2023-05-28|       9|  1314|    972|   1824|     1362|    Sun|
|  2023-05-29|      24|  2070|   3072|   3254|     3194|    Mon|
+------------+--------+------+-------+-------+---------+-------+

